## Kinematics Project Test notebook

In [10]:
from sympy import symbols, cos, sin, pi, sqrt, simplify, atan2,acos
from sympy.matrices import Matrix
import numpy as np

### Forward Kinematics
This is to get the forward kinematics, the convention is marked in the image below


![alt text](DH.png "DH annotation")


#### DH Parameters for Kuka arm
Following the DH Convention

| i        | a(i-1)  | a(i-1)  | d(i-1)  | θ(i-1)  |
| :------- |:-------:|:-------:|:-------:|:-------:|
| 1        | 0       | 0       | d1      |θ1       |
| 2        | -90     | a1      | 0       |θ2 - 90  |
| 3        | 0       | a2      | 0       |θ3       |
| 4        | -90     | a3      | d4      |θ4       |
| 5        | 90      | 0       | 0       |θ5       |
| 6        | -90     | 0       | 0       |θ6       |
| G        | 0       | 0       | dG      |0        |

#### Transformation matrix for Kuka arm


In [11]:
def rot_x(q):
    R_x = Matrix([[ 1,              0,        0],
              [ 0,        cos(q), -sin(q)],
              [ 0,        sin(q),  cos(q)]])
    
    return R_x
    
def rot_y(q):              
    R_y = Matrix([[ cos(q),        0,  sin(q)],
              [       0,        1,        0],
              [-sin(q),        0,  cos(q)]])
    
    return R_y

def rot_z(q):    
    R_z = Matrix([[ cos(q), -sin(q),        0],
              [ sin(q),  cos(q),        0],
              [ 0,              0,        1]])
    
    return R_z

In [12]:
# q = theta
def transformation_matrix(alpha,a,d,q):    
    return Matrix([[        cos(q),        -sin(q),       0,           a],
                   [ sin(q)*cos(alpha),  cos(q)*cos(alpha),  -sin(alpha),  -sin(alpha)*d],
                   [ sin(q)*sin(alpha),  cos(q)*sin(alpha),   cos(alpha),   cos(alpha)*d],
                   [             0,              0,        0,        1]])


This gets the transform matrices without any replacement of d and a


In [44]:
q1, q2, q3, q4, q5, q6 = symbols('q1:7') #theta
a0, a1, a2, a3, a4, a5 = symbols('a0:6')
d1, d2, d3, d4, d5, d6, dG = symbols('d1:8')
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5 = symbols('alpha0:6')

T1_0 = transformation_matrix(0, 0, d1, q1)
T2_1 = transformation_matrix(-pi/2, a1, 0, q2 - pi/2) #remember to offset q2 = -pi/2
T3_2 = transformation_matrix(0, a2, 0, q3)
T4_3 = transformation_matrix(-pi/2, a3, d4, q4)
T5_4 = transformation_matrix(pi/2, 0, 0, q5)
T6_5 = transformation_matrix(-pi/2, 0, 0, q6)

rG_6 = rot_z(pi) * rot_y(-pi/2)
tG_6 = Matrix([[0],[0],[dG]])
TG_6 = rG_6.row_join(tG_6)
TG_6 = TG_6.col_join(Matrix([[0,0,0,1]]))

print("T1_0 = \n", simplify(T1_0))
print("")
print("T2_1 = ", simplify(T2_1))
print("")
print("T3_2 = ", simplify(T3_2))
print("")
print("T4_3 = ", simplify(T4_3))
print("")
print("T5_4 = ", simplify(T5_4))
print("")
print("T6_5 = ", simplify(T6_5))
print("")
print("TG_6 = ", simplify(TG_6))

('T1_0 = \n', Matrix([
[cos(q1), -sin(q1), 0,  0],
[sin(q1),  cos(q1), 0,  0],
[      0,        0, 1, d1],
[      0,        0, 0,  1]]))

('T2_1 = ', Matrix([
[sin(q2),  cos(q2), 0, a1],
[      0,        0, 1,  0],
[cos(q2), -sin(q2), 0,  0],
[      0,        0, 0,  1]]))

('T3_2 = ', Matrix([
[cos(q3), -sin(q3), 0, a2],
[sin(q3),  cos(q3), 0,  0],
[      0,        0, 1,  0],
[      0,        0, 0,  1]]))

('T4_3 = ', Matrix([
[ cos(q4), -sin(q4), 0, a3],
[       0,        0, 1, d4],
[-sin(q4), -cos(q4), 0,  0],
[       0,        0, 0,  1]]))

('T5_4 = ', Matrix([
[cos(q5), -sin(q5),  0, 0],
[      0,        0, -1, 0],
[sin(q5),  cos(q5),  0, 0],
[      0,        0,  0, 1]]))

('T6_5 = ', Matrix([
[ cos(q6), -sin(q6), 0, 0],
[       0,        0, 1, 0],
[-sin(q6), -cos(q6), 0, 0],
[       0,        0, 0, 1]]))

('TG_6 = ', Matrix([
[0,  0, 1,  0],
[0, -1, 0,  0],
[1,  0, 0, d7],
[0,  0, 0,  1]]))


Getting the homogeneous transform matrix from base_link to gripper_link

In [45]:
T2_0 = T1_0*T2_1 #base_link to link_2

In [46]:
T3_0 = T2_0*T3_2 #link_2 to link_3

In [47]:
T4_0 = T3_0*T4_3 #link_3 to link_4

In [48]:
T5_0 = T4_0*T5_4 #link_4 to link_5

In [49]:
T6_0 = T5_0*T6_5 #link_5 to link_6
print("T6_0 = ", simplify(T6_0),"\n")

('T6_0 = ', Matrix([
[((sin(q1)*sin(q4) + sin(q2 + q3)*cos(q1)*cos(q4))*cos(q5) + sin(q5)*cos(q1)*cos(q2 + q3))*cos(q6) - (-sin(q1)*cos(q4) + sin(q4)*sin(q2 + q3)*cos(q1))*sin(q6), -((sin(q1)*sin(q4) + sin(q2 + q3)*cos(q1)*cos(q4))*cos(q5) + sin(q5)*cos(q1)*cos(q2 + q3))*sin(q6) + (sin(q1)*cos(q4) - sin(q4)*sin(q2 + q3)*cos(q1))*cos(q6), -(sin(q1)*sin(q4) + sin(q2 + q3)*cos(q1)*cos(q4))*sin(q5) + cos(q1)*cos(q5)*cos(q2 + q3), (a1 + a2*sin(q2) + a3*sin(q2 + q3) + d4*cos(q2 + q3))*cos(q1)],
[ ((sin(q1)*sin(q2 + q3)*cos(q4) - sin(q4)*cos(q1))*cos(q5) + sin(q1)*sin(q5)*cos(q2 + q3))*cos(q6) - (sin(q1)*sin(q4)*sin(q2 + q3) + cos(q1)*cos(q4))*sin(q6), -((sin(q1)*sin(q2 + q3)*cos(q4) - sin(q4)*cos(q1))*cos(q5) + sin(q1)*sin(q5)*cos(q2 + q3))*sin(q6) - (sin(q1)*sin(q4)*sin(q2 + q3) + cos(q1)*cos(q4))*cos(q6), -(sin(q1)*sin(q2 + q3)*cos(q4) - sin(q4)*cos(q1))*sin(q5) + sin(q1)*cos(q5)*cos(q2 + q3), (a1 + a2*sin(q2) + a3*sin(q2 + q3) + d4*cos(q2 + q3))*sin(q1)],
[                                

In [50]:
TG_0 = simplify(T6_0*TG_6) #link_6 to link_G
print("TG_0 = ", simplify(TG_0),"\n")

('TG_0 = ', Matrix([
[-(sin(q1)*sin(q4) + sin(q2 + q3)*cos(q1)*cos(q4))*sin(q5) + cos(q1)*cos(q5)*cos(q2 + q3), ((sin(q1)*sin(q4) + sin(q2 + q3)*cos(q1)*cos(q4))*cos(q5) + sin(q5)*cos(q1)*cos(q2 + q3))*sin(q6) - (sin(q1)*cos(q4) - sin(q4)*sin(q2 + q3)*cos(q1))*cos(q6), ((sin(q1)*sin(q4) + sin(q2 + q3)*cos(q1)*cos(q4))*cos(q5) + sin(q5)*cos(q1)*cos(q2 + q3))*cos(q6) + (sin(q1)*cos(q4) - sin(q4)*sin(q2 + q3)*cos(q1))*sin(q6), a1*cos(q1) + a2*sin(q2)*cos(q1) + a3*sin(q2 + q3)*cos(q1) + d4*cos(q1)*cos(q2 + q3) - d7*((sin(q1)*sin(q4) + sin(q2 + q3)*cos(q1)*cos(q4))*sin(q5) - cos(q1)*cos(q5)*cos(q2 + q3))],
[-(sin(q1)*sin(q2 + q3)*cos(q4) - sin(q4)*cos(q1))*sin(q5) + sin(q1)*cos(q5)*cos(q2 + q3), ((sin(q1)*sin(q2 + q3)*cos(q4) - sin(q4)*cos(q1))*cos(q5) + sin(q1)*sin(q5)*cos(q2 + q3))*sin(q6) + (sin(q1)*sin(q4)*sin(q2 + q3) + cos(q1)*cos(q4))*cos(q6), ((sin(q1)*sin(q2 + q3)*cos(q4) - sin(q4)*cos(q1))*cos(q5) + sin(q1)*sin(q5)*cos(q2 + q3))*cos(q6) - (sin(q1)*sin(q4)*sin(q2 + q3) + cos(q1)*co

Replacing it with d and a from Kuka arm 210 xacro

In [51]:
# DH Parameters
# Get original position to test the results
s = {alpha0:     0,  a0:      0, d1: 0.75, q1: 0,
     alpha1: -pi/2,  a1:   0.35, d2:    0, q2: 0,
     alpha2:     0,  a2:   1.25, d3:    0, q3:0,
     alpha3: -pi/2,  a3: -0.054, d4:  1.5, q4:0,
     alpha4:  pi/2,  a4:      0, d5:    0, q5: 0,
     alpha5: -pi/2,  a5:      0, d6:    0, q6:0,
     dG: 0.303 }

E_G = TG_0.evalf(subs=s, chop = True)
print(simplify(E_G))
E_6 = T6_0.evalf(subs=s, chop = True)
print(simplify(E_6))

E_5 = T5_0.evalf(subs=s, chop = True)
print(simplify(E_5))
E_4 = T4_0.evalf(subs=s, chop = True)
print(simplify(E_4))
E_3 = T3_0.evalf(subs=s, chop = True)
print(simplify(E_3))

Matrix([
[1.0,   0,   0, 2.153],
[  0, 1.0,   0,     0],
[  0,   0, 1.0, 1.946],
[  0,   0,   0,   1.0]])
Matrix([
[  0,    0, 1.0,  1.85],
[  0, -1.0,   0,     0],
[1.0,    0,   0, 1.946],
[  0,    0,   0,   1.0]])
Matrix([
[  0, 1.0,   0,  1.85],
[  0,   0, 1.0,     0],
[1.0,   0,   0, 1.946],
[  0,   0,   0,   1.0]])
Matrix([
[  0,    0, 1.0,  1.85],
[  0, -1.0,   0,     0],
[1.0,    0,   0, 1.946],
[  0,    0,   0,   1.0]])
Matrix([
[  0, 1.0,   0, 0.35],
[  0,   0, 1.0,    0],
[1.0,   0,   0,  2.0],
[  0,   0,   0,  1.0]])


### Inverse Kinematics
This is to get the forward kinematics

Solving the first three equations of wrist center.
Location of the WC relative to the base frame

![alt text](IK_WC.png "IK: Wrist center")

r13, r23, and r33 define the Z-axis of the EE relative to the base frame, the Cartesian coordinates of the WC is

    Wx = Px - (d6 + l)*nx
    Wy = Py - (d6 + l)*ny
    Wz = Pz - (d6 + l)*nz

Where,
Px, Py, Pz = end-effector positions
Wx, Wy, Wz = wrist positions
d6 = from DH table
l = end-effector length

![alt text](Joint.png "IK: Joint ")

The rotation for the speherical arm is as below, it provides the roll pitch yaw 

![alt text](rot_spherical.png "IK: rot_spherical ")

For the arm it is translated along the x-axis (according to RViz). thus it would be R * [1 0 0], where we would need r11,r21,r31

    Lx = r11 = cos(alpha)*cos(beta)
    Ly = r21 = sin(alpha)*cos(beta)
    Lz = r31 = -sin(beta)

Convention: L = R [1, 0, 0]; M = R [0, 1, 0]; N = R [ 0, 0, 1]

In [148]:
# this is to test the IK for q2 and q3

px, py, pz, nx, ny, nz = symbols("px py pz nx ny nz")

#link between joint 2 to 3
l2_l3 = 1.25
#link between joint 3 to wc
l3_wc = sqrt(1.5*1.5 + 0.054*0.054)
#distance in x axis between base link and joint 2
dx0_1 = 0.35
#distance in z axis between base link and joint 2
dz0_1 = 0.75
#distance in gripper to wc
dG_wc = 0.303

#calculating wc
wx = px - (dG_wc)*nx
wy = py - (dG_wc)*ny
wz = pz - (dG_wc)*nz

xc = (sqrt(wx**2 + wy**2) - dx0_1)
zc = (wz - dz0_1)
theta1 = atan2(wy,wx)

r = (xc*xc + zc*zc - l3_wc*l3_wc - l2_l3*l2_l3)/(2*l3_wc*l2_l3)
theta3 = atan2(sqrt(1-r*r), r)
theta3b = atan2(-sqrt(1-r*r), r) 
theta2 = atan2(xc,zc) - atan2(l3_wc*sin(theta3), l2_l3+l3_wc*cos(theta3))
theta2b = atan2(xc,zc) - atan2(l3_wc*sin(theta3b), l2_l3+l3_wc*cos(theta3b))

In [149]:
# Testing: get IK

# ik_1 = {px: 0.8, py: 1.8, pz: 1.5, nx: 1.5, ny: 0.2, nz: -0.8}
# ik_1 = {px: 2.089, py: -0.9, pz: 1.58, nx: 0.99, ny: 0.00054, nz:-0.0003225}

ik_1 = {px: 2.153, py: 0, pz: 1.946, nx: 1, ny: 0, nz:0}

ik_wx = wx.evalf(subs=ik_1, chop = True)
ik_wy = wy.evalf(subs=ik_1, chop = True)
ik_wz = wz.evalf(subs=ik_1, chop = True)

print "wx:", ik_wx, " wy: ", ik_wy, " wz: ", ik_wz

ik_q1 = theta1.evalf(subs=ik_1, chop = True)
ik_q3 = theta3.evalf(subs=ik_1, chop = True)
ik_q2 = theta2.evalf(subs=ik_1, chop = True)
ik_q3b = theta3b.evalf(subs=ik_1, chop = True)
ik_q2b = theta2b.evalf(subs=ik_1, chop = True)

ik_q3= (ik_q3 - np.pi/2)
ik_q3b= (ik_q3b - np.pi/2)

print "ik_q1:", ik_q1, " ik_q2: ", ik_q2, " ik_q3: ", ik_q3
print "ik_q1:", ik_q1, " ik_q2b: ", ik_q2b, " ik_q3b: ", ik_q3b

wx: 1.85000000000000  wy:  0  wz:  1.94600000000000
ik_q1: 0  ik_q2:  2.36418021617164e-16  ik_q3:  0.0359844600820514
ik_q1: 0  ik_q2b:  1.79536703458194  ik_q3b:  -3.17757711367184


In [153]:
# Testing: FK with received IK

l3 = sqrt(0.96*0.96 + 0.054*0.054)
mod_a3 = -0.054 # revisit: l3*sin(ik_q3)
mod_d4 = 1.5 # revisit:  0.54 + l3*cos(ik_q3)
print "mod_d4: ", mod_d4, " mod_a3: ", mod_a3

s = {alpha0:     0,  a0:      0, d1: 0.75, q1: ik_q1,
     alpha1: -pi/2,  a1:   0.35, d2:    0, q2: ik_q2,
     alpha2:     0,  a2:   1.25, d3:    0, q3: ik_q3, 
     alpha3: -pi/2,  a3: mod_a3, d4:  mod_d4, q4:0,
     alpha4:  pi/2,  a4:      0, d5:    0, q5: 0,
     alpha5: -pi/2,  a5:      0, d6:    0, q6:0,
     dG: 0.303 }

# taking position 4 and 5 are the same, which should be the Wc
E_5 = T5_0.evalf(subs=s, chop = True)
print(simplify(E_5))

s = {alpha0:     0,  a0:      0, d1: 0.75, q1: ik_q1,
     alpha1: -pi/2,  a1:   0.35, d2:    0, q2: ik_q2b,
     alpha2:     0,  a2:   1.25, d3:    0, q3: ik_q3b,
     alpha3: -pi/2,  a3: mod_a3, d4:  mod_d4, q4:0,
     alpha4:  pi/2,  a4:      0, d5:    0, q5: 0,
     alpha5: -pi/2,  a5:      0, d6:    0, q6:0,
     dG: 0.303 }

# taking position 4 and 5 are the same, which should be the Wc
E_5b = T5_0.evalf(subs=s, chop = True)
print(simplify(E_5b))

mod_d4:  1.5  mod_a3:  -0.054
Matrix([
[0.0359766946503549,   0.999352629176526,   0, 1.84708620225367],
[                 0,                   0, 1.0,                0],
[ 0.999352629176526, -0.0359766946503549,   0, 1.89206991604894],
[                 0,                   0,   0,              1.0]])
Matrix([
[-0.982270253360949, 0.187470395962179,   0, 1.90286043145807],
[                 0,                 0, 1.0,                0],
[ 0.187470395962179, 0.982270253360949,   0, 1.93492214107589],
[                 0,                 0,   0,              1.0]])


In [154]:
# Testing: get IK

ik_2 = {px: 2.089, py: -0.9, pz: 1.58, nx: 0.99, ny: 0.00054, nz:-0.0003225}

ik_wx = wx.evalf(subs=ik_2, chop = True)
ik_wy = wy.evalf(subs=ik_2, chop = True)
ik_wz = wz.evalf(subs=ik_2, chop = True)

print "wx:", ik_wx, " wy: ", ik_wy, " wz: ", ik_wz


ik_q1 = theta1.evalf(subs=ik_2, chop = True)
ik_q3 = theta3.evalf(subs=ik_2, chop = True)
ik_q2 = theta2.evalf(subs=ik_2, chop = True)
ik_q3b = theta3b.evalf(subs=ik_2, chop = True)
ik_q2b = theta2b.evalf(subs=ik_2, chop = True)

ik_q3= (ik_q3 - np.pi/2)
ik_q3b= (ik_q3b - np.pi/2)

print "ik_q1:", ik_q1, " ik_q2: ", ik_q2, " ik_q3: ", ik_q3
print "ik_q1:", ik_q1, " ik_q2b: ", ik_q2b, " ik_q3b: ", ik_q3b

# Testing: FK with received IK
s = {alpha0:     0,  a0:      0, d1: 0.75, q1: ik_q1,
     alpha1: -pi/2,  a1:   0.35, d2:    0, q2: ik_q2,
     alpha2:     0,  a2:   1.25, d3:    0, q3: ik_q3, # offset by -pi/2
     alpha3: -pi/2,  a3: mod_a3, d4:  mod_d4, q4:0,
     alpha4:  pi/2,  a4:      0, d5:    0, q5: 0,
     alpha5: -pi/2,  a5:      0, d6:    0, q6:0,
     dG: 0.303 }

# taking position 4 and 5 are the same, which should be the Wc
E_5 = T5_0.evalf(subs=s, chop = True)
print(simplify(E_5))

s = {alpha0:     0,  a0:      0, d1: 0.75, q1: ik_q1,
     alpha1: -pi/2,  a1:   0.35, d2:    0, q2: ik_q2b,
     alpha2:     0,  a2:   1.25, d3:    0, q3: ik_q3b, # offset by -pi/2
     alpha3: -pi/2,  a3: mod_a3, d4:  mod_d4, q4:0,
     alpha4:  pi/2,  a4:      0, d5:    0, q5: 0,
     alpha5: -pi/2,  a5:      0, d6:    0, q6:0,
     dG: 0.303 }

# taking position 4 and 5 are the same, which should be the Wc
E_5b = T5_0.evalf(subs=s, chop = True)
print(simplify(E_5b))

wx: 1.78903000000000  wy:  -0.900163620000000  wz:  1.58009771750000
ik_q1: -0.466170309293007  ik_q2:  0.166188898637931  ik_q3:  0.105417939069464
ik_q1: -0.466170309293007  ik_q2b:  2.04449762288901  ik_q3b:  -3.24701059265926
Matrix([
[ 0.23965323937271,  0.860548868330463, 0.449468541806127,  1.77525254240932],
[-0.12058329234192, -0.432991500703316, 0.893296160255194, -0.89323139074771],
[0.963341058227121, -0.268279715099465,                 0,  1.52833798380472],
[                0,                  0,                 0,               1.0]])
Matrix([
[-0.833397737145082,  0.321599505049572, 0.449468541806127,   1.83372098430204],
[ 0.419330209090024, -0.161815159419144, 0.893296160255194, -0.922650217883035],
[ 0.360014426747002,  0.932946736172021,                 0,   1.55975040784892],
[                 0,                  0,                 0,                1.0]])


In [155]:
# Testing: get IK

ik_3 = {px: 0.8, py: 1.8, pz: 1.5, nx: 1.5, ny: 0.2, nz: -0.8}

ik_wx = wx.evalf(subs=ik_3, chop = True)
ik_wy = wy.evalf(subs=ik_3, chop = True)
ik_wz = wz.evalf(subs=ik_3, chop = True)

print "wx:", ik_wx, " wy: ", ik_wy, " wz: ", ik_wz

ik_q1 = theta1.evalf(subs=ik_3, chop = True)
ik_q3 = theta3.evalf(subs=ik_3, chop = True)
ik_q2 = theta2.evalf(subs=ik_3, chop = True)
ik_q3b = theta3b.evalf(subs=ik_3, chop = True)
ik_q2b = theta2b.evalf(subs=ik_3, chop = True)

ik_q3= (ik_q3 - np.pi/2)
ik_q3b= (ik_q3b - np.pi/2)

print "ik_q1:", ik_q1, " ik_q2: ", ik_q2, " ik_q3: ", ik_q3
print "ik_q1:", ik_q1, " ik_q2b: ", ik_q2b, " ik_q3b: ", ik_q3b

# Testing: FK with received IK
s = {alpha0:     0,  a0:      0, d1: 0.75, q1: ik_q1,
     alpha1: -pi/2,  a1:   0.35, d2:    0, q2: ik_q2,
     alpha2:     0,  a2:   1.25, d3:    0, q3: ik_q3, # offset by -pi/2
     alpha3: -pi/2,  a3: mod_a3, d4:  mod_d4, q4:0,
     alpha4:  pi/2,  a4:      0, d5:    0, q5: 0,
     alpha5: -pi/2,  a5:      0, d6:    0, q6:0,
     dG: 0.303 }

# taking position 4 and 5 are the same, which should be the Wc
E_5 = T5_0.evalf(subs=s, chop = True)
print(simplify(E_5))

s = {alpha0:     0,  a0:      0, d1: 0.75, q1: ik_q1,
     alpha1: -pi/2,  a1:   0.35, d2:    0, q2: ik_q2b,
     alpha2:     0,  a2:   1.25, d3:    0, q3: ik_q3b, # offset by -pi/2
     alpha3: -pi/2,  a3: mod_a3, d4:  mod_d4, q4:0,
     alpha4:  pi/2,  a4:      0, d5:    0, q5: 0,
     alpha5: -pi/2,  a5:      0, d6:    0, q6:0,
     dG: 0.303 }

# taking position 4 and 5 are the same, which should be the Wc
E_5b = T5_0.evalf(subs=s, chop = True)
print(simplify(E_5b))

wx: 0.345500000000000  wy:  1.73940000000000  wz:  1.74240000000000
ik_q1: 1.37471677392652  ik_q2:  -0.0444420940218326  ik_q3:  0.216083502962604
ik_q1: 1.37471677392652  ik_q2b:  1.96832886336129  ik_q3b:  -3.35767615655240
Matrix([
[0.0332761722047489,  0.191962698278435, -0.980837916698301, 0.343516559373503],
[ 0.167526986781303,  0.966425231217107,  0.194825514671302,  1.72941448154637],
[ 0.985305741921449, -0.170799868081398,                  0,  1.68935945365318],
[                 0,                  0,                  0,               1.0]])
Matrix([
[ -0.19162711097915, 0.0351572397198971, -0.980837916698301, 0.355813702220697],
[-0.964735736142209,  0.176997113657855,  0.194825514671302,  1.79132374426246],
[ 0.180455007544634,  0.983583240123614,                  0,  1.73169969624053],
[                 0,                  0,                  0,               1.0]])


In [ ]:
# To obtain q4,5,6
Rrpy = R6_0
R6_3 = R3_0.inv() * Rrpy
print(simplify(R6_3))